In [4]:
import random
import pickle
import os
import cv2
import numpy as np
import tensorflow
from PIL import Image
from matplotlib import pyplot as plt
from patchify import patchify
from enum import Enum
import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score
from predict_B_fun import predict_modelB
from predict_A_fun import predict_modelA


In [5]:
scale = 1
# enlarge pixel size
# eg. scale 4 changes pixel size from 25x25cm to 100x100cm

chosen_model = 2
# 1 for Model A
# 2 for Model B
# 3 for Model C

In [6]:
# root_directory = r"C:\Users\jiric\Documents\VUT\BP\DATASETS\DUBAI_DS\Semantic segmentation dataset"
root_directory = r"C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test"
# root_directory = r"C:\Users\jiric\Documents\VUT\BP\DATASETS\LANDCOVER_AI\ORIGINAL"

image_paths = []
mask_paths = []

for path, subdirs, files in sorted(os.walk(root_directory)):
    dirname = path.split(os.path.sep)[-1]
    if dirname == "images":
        image_paths.extend([os.path.join(path, img_name) for img_name in sorted(files)])
    elif dirname == "masks":
        mask_paths.extend([os.path.join(path, mask_name) for mask_name in sorted(files)])


print("Image paths:")
for img_path in image_paths:
    print(img_path)

print("\nMask paths:")
for mask_path in mask_paths:
    print(mask_path)



Image paths:
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test\images\10378780_15.tiff
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test\images\10828720_15.tiff
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test\images\11128870_15.tiff
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test\images\11278840_15.tiff
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test\images\11728825_15.tiff
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test\images\12328750_15.tiff
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test\images\15928855_15.tiff
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSETTS_ROADS_DATASET\upravena struktura test\images\16078870_15.tiff
C:\Users\jiric\Documents\VUT\BP\DATASETS\MASSACHUSE

In [7]:
print(len(image_paths))
print(len(mask_paths))

if len(image_paths) == len(mask_paths):
    print("Lists of images and masks paths consist of the same number of elements:", len(image_paths))
else:
    print("ERROR: Different number of images and masks")

paths_length = len(image_paths)

accuracy_values = []
jaccard_values = []

for i in range(paths_length):

    print("\n IMAGE",i+1,"/",paths_length,"\n")
    
    mask_path = mask_paths[i]
    image_path = image_paths[i]

    if chosen_model == 1:
        image_original, image_cropped, image_prediction = predict_modelA(image_path,scale)
    elif chosen_model == 2:
        image_original, image_cropped, image_prediction = predict_modelB(image_path,scale)
    elif chosen_model == 3:
        image_original, image_cropped, image_prediction = predict_modelC(image_path,scale)
    else:
        print("ERROR: INVALID MODEL CHOSEN")


    mask_original = cv2.cvtColor(cv2.imread(mask_path), cv2.COLOR_BGR2RGB)
    mask_scaled = cv2.resize(mask_original, (mask_original.shape[1] // scale, mask_original.shape[0] // scale))

    new_height_m = image_cropped.shape[0]
    new_width_m = image_cropped.shape[1]

    mask_cropped = mask_scaled[: new_height_m, : new_width_m, :]

    # PROCESOVANI PRO MASKY DATASETU MASSACHUSETTS
    mask_cropped = (mask_cropped[:,:,0] < 128).astype(np.float32)
    mask_cropped = np.expand_dims(mask_cropped, axis=-1)

    print("Cropped mask shape:", mask_cropped.shape)
    print("Cropped mask values", np.unique(mask_cropped))

    if (mask_cropped.shape[:] == image_cropped.shape[:]):
        print("MASK AND IMAGE SHAPE CONSISTENT")
    else:
        print("ERROR: DIFFERENT SHAPE FOR MASK AND IMAGE")

    mask_flat = mask_cropped.flatten()
    pred_flat = image_prediction.flatten()

    sk_acc = accuracy_score(mask_flat, pred_flat)
    sk_jac = jaccard_score(mask_flat, pred_flat, average="macro")

    accuracy_values.append(sk_acc)
    jaccard_values.append(sk_jac)

    print("Prediction accuracy:", sk_acc)
    print("Prediction jaccard index:", sk_jac)

    

49
49
Lists of images and masks paths consist of the same number of elements: 49

 IMAGE 1 / 49 

Original image shape: (1500, 1500, 3)
FINAL TIME: 2.852311849594116
Predicted image shape: (1280, 1280, 1)
Cropped mask shape: (1280, 1280, 1)
Cropped mask values [0. 1.]
ERROR: DIFFERENT SHAPE FOR MASK AND IMAGE
Prediction accuracy: 0.985948486328125
Prediction jaccard index: 0.7603649976095532

 IMAGE 2 / 49 

Original image shape: (1500, 1500, 3)
FINAL TIME: 3.0592257976531982
Predicted image shape: (1280, 1280, 1)
Cropped mask shape: (1280, 1280, 1)
Cropped mask values [0. 1.]
ERROR: DIFFERENT SHAPE FOR MASK AND IMAGE
Prediction accuracy: 0.9580230712890625
Prediction jaccard index: 0.7005757870879934

 IMAGE 3 / 49 

Original image shape: (1500, 1500, 3)
FINAL TIME: 2.4241857528686523
Predicted image shape: (1280, 1280, 1)
Cropped mask shape: (1280, 1280, 1)
Cropped mask values [0. 1.]
ERROR: DIFFERENT SHAPE FOR MASK AND IMAGE
Prediction accuracy: 0.9724798583984375
Prediction jaccard

KeyboardInterrupt: 

In [ ]:
print(len(accuracy_values))
print(len(jaccard_values))

mean_j = np.mean(jaccard_values)
mean_a = np.mean(accuracy_values)

print(mean_a)
print(mean_j)

idm = jaccard_values.index(max(jaccard_values))
print(idm)

In [ ]:
idw = random.randint(0, len(accuracy_values) - 1)

if chosen_model == 1:
    _ , image_sample, prediction_sample = predict_modelA(image_paths[idw],scale)
elif chosen_model == 2:
    _ , image_sample, prediction_sample = predict_modelB(image_paths[idw],scale)

mask_sample = cv2.imread(mask_paths[idw])
mask_sample = cv2.cvtColor(mask_sample, cv2.COLOR_BGR2RGB)
new_height_m = image_cropped.shape[0]
new_width_m = image_cropped.shape[1]

mask_scaled = cv2.resize(mask_sample, (mask_sample.shape[1] // scale, mask_sample.shape[0] // scale))

new_height_m = prediction_sample.shape[0]
new_width_m = prediction_sample.shape[1]

mask_sample = mask_scaled[: new_height_m, : new_width_m, :]

print("Cropped mask shape: ", mask_sample.shape)


print("ACCURACY = ", accuracy_values[idw])
print("JACCARD INDEX = ", jaccard_values[idw])

plt.figure(figsize=(15, 5))

plt.subplot(1,3,1)
plt.imshow(cv2.cvtColor(image_sample, cv2.COLOR_BGR2RGB))
plt.title('Cropped image')
plt.axis('off')

plt.subplot(1,3,2)
plt.imshow(mask_sample)
plt.title('Cropped mask')
plt.axis('off')

plt.subplot(1,3,3)
plt.imshow(prediction_sample, cmap='binary')
plt.title('Prediction')
plt.axis('off')

plt.show()